In [ ]:
import os
import openai
import json

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai == None:
    print("OPENAI_API_KEY environment variable not set")
    openai_api_key = input("Enter your API key: ")
openai.api_key = openai_api_key

In [ ]:
model_list = openai.Model.list()
print([m["id"] for m in dict(model_list)["data"]])
standard_model = "text-davinci-003"

In [ ]:
test_response = openai.Completion.create(
    model=standard_model, 
    prompt="Question: \"Say OK if you understand.\"\nAnswer: \"", 
    max_tokens=3,
    stop=["\""])

In [ ]:
print(test_response["choices"][0]["text"])

In [ ]:
def ask_for_stackoverflow_answer(question_title, question_body):
    prompt = f"""QUESTION TITLE: {question_title}\nQUESTION BODY:{question_body}\n\nANSWER:"""
    print(prompt)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful StackOverflow user. You are answering a question on StackOverflow."},
            {"role": "user", "content": "I want you to answer a Question I found on StackOverflow. You must answer using the StackOverflow formatting rules:\n**bold**\n*italics*\n`code`"},
            {"role": "system", "content": "I will answer you questions using in a typical StackOverflow style using the provided formatting rules."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000
    )

    return response.choices[0].message.content

In [ ]:
def ask_for_stackoverflow_answer_completion(question_title, question_body):
    prompt = f"""Question on StackOverflow titled \"{question_title}\":\n{question_body}\n\n""" + """A short and consice answer in typical StackOverflow manner, using the required formatting for code snippets for StackOverflow answers:\n"""
    print(prompt)
    response = openai.Completion.create(
        model=standard_model, 
        prompt=prompt,
        max_tokens=1000)

    return response["choices"][0]["text"]

In [ ]:
# answer = ask_for_stackoverflow_answer("How do I get the current date in Python?","I have a Python script and I want to display today's date in YYYY-MM-DD format. How can I do that?")
# print(answer)
